In [153]:
# Library imports

from __future__ import annotations

from dataclasses import dataclass, asdict
from pathlib import Path
import hashlib
import json
import numpy as np
import pandas as pd
import pyarrow.dataset as ps
import glob
import os
import matplotlib.pyplot as plt
import random
import time
import math
import logging
import itertools

from openpyxl import load_workbook
import xlsxwriter as xw

In [154]:
@dataclass
class CommunityProfile:
    name: str
    load_profile: pd.DataFrame

In [155]:
# Define the file pattern to search for the RESSTOCK *.Parquet files
DATA_DIR = "../data/"

# Define input data directories
INPUT_DATA_DIR = os.path.join(DATA_DIR, "input/")

# RESSTOCK directories
INPUT_DATA_DIR_RESSTOCK = os.path.join(INPUT_DATA_DIR, "resstock/")
INPUT_DATA_DIR_RESSTOCK_BUILDINGPROFILES = os.path.join(INPUT_DATA_DIR_RESSTOCK, "timeseries_individual_buildings/")

# Define output data directories

OUTPUT_DATA_DIR = os.path.join(DATA_DIR, "output/")
OUTPUT_DATA_DIR_SCENARIO_RUNS = os.path.join(OUTPUT_DATA_DIR, "scenario_runs/")

In [156]:
# COMMSTOCK directories
INPUT_DATA_DIR_COMSTOCK = os.path.join(INPUT_DATA_DIR, "comstock/")
INPUT_DATA_DIR_COMSTOCK_BUILDINGPROFILES = os.path.join(INPUT_DATA_DIR_COMSTOCK, "timeseries_individual_buildings/")

In [157]:
# "0" is baseline scenario
UPGRADE_NUM = "0"

# Washington, DC
STATE = "DC"

file_pattern = os.path.join(INPUT_DATA_DIR_RESSTOCK_BUILDINGPROFILES, f"upgrade={UPGRADE_NUM}/state={STATE}/", "*.parquet")

# Get a list of all file names matching the pattern
all_files = glob.glob(file_pattern, recursive=True)

print(f"Total RESSTOCK files found: {all_files}")

Total RESSTOCK files found: ['../data/input/resstock/timeseries_individual_buildings/upgrade=0/state=DC/206093-0.parquet', '../data/input/resstock/timeseries_individual_buildings/upgrade=0/state=DC/79081-0.parquet', '../data/input/resstock/timeseries_individual_buildings/upgrade=0/state=DC/541915-0.parquet', '../data/input/resstock/timeseries_individual_buildings/upgrade=0/state=DC/130440-0.parquet', '../data/input/resstock/timeseries_individual_buildings/upgrade=0/state=DC/89323-0.parquet', '../data/input/resstock/timeseries_individual_buildings/upgrade=0/state=DC/371093-0.parquet', '../data/input/resstock/timeseries_individual_buildings/upgrade=0/state=DC/398150-0.parquet', '../data/input/resstock/timeseries_individual_buildings/upgrade=0/state=DC/518977-0.parquet', '../data/input/resstock/timeseries_individual_buildings/upgrade=0/state=DC/86497-0.parquet', '../data/input/resstock/timeseries_individual_buildings/upgrade=0/state=DC/77991-0.parquet', '../data/input/resstock/timeseries_

In [158]:
RESSTOCK_BUILDING_CHARACTERISTICS_FILE = 'DC_upgrade0.xlsx'
RESSTOCK_BUILDING_CHARACTERISTICS_FILEPATH = DATA_DIR + 'background/' + RESSTOCK_BUILDING_CHARACTERISTICS_FILE
building_characteristics = pd.read_excel(RESSTOCK_BUILDING_CHARACTERISTICS_FILEPATH, sheet_name='building_characteristics')


In [159]:
## Build the neighborhood

TOTAL_BUILDINGS = 100 # could be adjusted or varied for additional analysis of different community sizes and their ability to utilize a microgrid

MULTIFAMILY_BUILDINGS_PERCENT_OF_TOTAL = 0.40
SINGLEFAMILY_ATTACHED_BUILDINGS_PERCENT_OF_TOTAL = 0.40
SINGLEFAMILY_DETACHED_BUILDINGS_PERCENT_OF_TOTAL = 0.20

if MULTIFAMILY_BUILDINGS_PERCENT_OF_TOTAL + SINGLEFAMILY_ATTACHED_BUILDINGS_PERCENT_OF_TOTAL + SINGLEFAMILY_DETACHED_BUILDINGS_PERCENT_OF_TOTAL != 1.0:
    raise ValueError("The sum of building type percentages must equal 1.0")

MULTIFAMILY_BUILDINGS = {
    "PERCENT_OF_TOTAL": MULTIFAMILY_BUILDINGS_PERCENT_OF_TOTAL,
    "SMALL": {
        "NAME" : ["Multi-Family with 2 - 4 Units", "Multi-Family with 5+ Units, 1-3 Stories"],
        "MULTIFAMILY_BUILDINGS_SMALL_PERCENT_OF_TOTAL": 0.75,
    },
    "MID" : {
        "NAME" : ["Multi-Family with 5+ Units, 4-7 Stories",],
        "MULTIFAMILY_BUILDINGS_MID_PERCENT_OF_TOTAL": 0.25,
    },
    "LARGE" : { 
        "NAME" : ["Multi-Family with 5+ Units, 8+ Stories",],
        "MULTIFAMILY_BUILDINGS_LARGE_PERCENT_OF_TOTAL": 0.0,
    },
}

SINGLEFAMILY_CHARACTERISTICS = {
    "AREALIMIT": 2000,
    "GARAGE": "None",
    "HEATPUMP": 0.1,
    #"PV": 0.2,
    #"EV": 0.2,
}

SINGLEFAMILY_ATTACHED_BUILDINGS = {
    "NAME": "Single-Family Attached",
    "PERCENT_OF_TOTAL": SINGLEFAMILY_ATTACHED_BUILDINGS_PERCENT_OF_TOTAL,
    "CHARACTERISTICS": SINGLEFAMILY_CHARACTERISTICS,
}
SINGLEFAMILY_DETACHED_BUILDINGS = {
    "NAME" : "Single-Family Detached",
    "PERCENT_OF_TOTAL" : SINGLEFAMILY_DETACHED_BUILDINGS_PERCENT_OF_TOTAL,
    "CHARACTERISTICS": SINGLEFAMILY_CHARACTERISTICS,
}


In [160]:

## construct multifamily

def construct_neighborhood_residential():

    multifamily_small_list = []
    mutlifamily_mid_list = []
    multifamily_large_list = []

    multifamily_small_list.extend(building_characteristics[building_characteristics['in.geometry_building_type_height'] == MULTIFAMILY_BUILDINGS["SMALL"]["NAME"][0]]['bldg_id'].tolist())
    multifamily_small_list.extend(building_characteristics[building_characteristics['in.geometry_building_type_height'] == MULTIFAMILY_BUILDINGS["SMALL"]["NAME"][1]]['bldg_id'].tolist())
    mutlifamily_mid_list.extend(building_characteristics[building_characteristics['in.geometry_building_type_height'] == MULTIFAMILY_BUILDINGS["MID"]["NAME"][0]]['bldg_id'].tolist())
    multifamily_large_list.extend(building_characteristics[building_characteristics['in.geometry_building_type_height'] == MULTIFAMILY_BUILDINGS["LARGE"]["NAME"][0]]['bldg_id'].tolist())

    multifamily_small_chosen = random.choices(multifamily_small_list, k=math.ceil(TOTAL_BUILDINGS * MULTIFAMILY_BUILDINGS["PERCENT_OF_TOTAL"] * MULTIFAMILY_BUILDINGS["SMALL"]["MULTIFAMILY_BUILDINGS_SMALL_PERCENT_OF_TOTAL"]))
    multifamily_mid_chosen = random.choices(mutlifamily_mid_list, k=math.ceil(TOTAL_BUILDINGS * MULTIFAMILY_BUILDINGS["PERCENT_OF_TOTAL"] * MULTIFAMILY_BUILDINGS["MID"]["MULTIFAMILY_BUILDINGS_MID_PERCENT_OF_TOTAL"]))
    multifamily_large_chosen = random.choices(multifamily_large_list, k=math.ceil(TOTAL_BUILDINGS * MULTIFAMILY_BUILDINGS["PERCENT_OF_TOTAL"] * MULTIFAMILY_BUILDINGS["LARGE"]["MULTIFAMILY_BUILDINGS_LARGE_PERCENT_OF_TOTAL"]))

    ## construct single family attached and detached

    singlefamily_attached_hp = building_characteristics[building_characteristics['in.geometry_garage'].isnull() 
                                                    & (building_characteristics['in.geometry_building_type_height'] == SINGLEFAMILY_ATTACHED_BUILDINGS["NAME"]) 
                                                    & (building_characteristics['in.sqft..ft2'] <= SINGLEFAMILY_ATTACHED_BUILDINGS["CHARACTERISTICS"]["AREALIMIT"])
                                                    & (building_characteristics['in.hvac_heating_type'] == 'Ducted Heat Pump')
                                                    | (building_characteristics['in.hvac_heating_type'] == 'Non-Ducted Heat Pump')]

    singlefamily_attached_nohp = building_characteristics[building_characteristics['in.geometry_garage'].isnull() 
                                                        & (building_characteristics['in.geometry_building_type_height'] == SINGLEFAMILY_ATTACHED_BUILDINGS["NAME"]) 
                                                        & (building_characteristics['in.sqft..ft2'] <= SINGLEFAMILY_ATTACHED_BUILDINGS["CHARACTERISTICS"]["AREALIMIT"])
                                                        & (building_characteristics['in.hvac_heating_type'] != 'Ducted Heat Pump')
                                                        & (building_characteristics['in.hvac_heating_type'] != 'Non-Ducted Heat Pump')]

    singlefamily_detached_hp = building_characteristics[building_characteristics['in.geometry_garage'].isnull() 
                                                    & (building_characteristics['in.geometry_building_type_height'] == SINGLEFAMILY_DETACHED_BUILDINGS["NAME"]) 
                                                    & (building_characteristics['in.sqft..ft2'] <= SINGLEFAMILY_DETACHED_BUILDINGS["CHARACTERISTICS"]["AREALIMIT"])
                                                    & (building_characteristics['in.hvac_heating_type'] == 'Ducted Heat Pump')
                                                    | (building_characteristics['in.hvac_heating_type'] == 'Non-Ducted Heat Pump')]

    singlefamily_detached_nohp = building_characteristics[building_characteristics['in.geometry_garage'].isnull() 
                                                        & (building_characteristics['in.geometry_building_type_height'] == SINGLEFAMILY_DETACHED_BUILDINGS["NAME"]) 
                                                        & (building_characteristics['in.sqft..ft2'] <= SINGLEFAMILY_DETACHED_BUILDINGS["CHARACTERISTICS"]["AREALIMIT"])
                                                        & (building_characteristics['in.hvac_heating_type'] != 'Ducted Heat Pump')
                                                        & (building_characteristics['in.hvac_heating_type'] != 'Non-Ducted Heat Pump')]

    singlefamily_attached_hp_list = singlefamily_attached_hp['bldg_id'].tolist()
    singlefamily_attached_nohp_list = singlefamily_attached_nohp['bldg_id'].tolist()
    singlefamily_detached_hp_list = singlefamily_detached_hp['bldg_id'].tolist()
    singlefamily_detached_nohp_list = singlefamily_detached_nohp['bldg_id'].tolist()

    singlefamily_attached_hp_chosen = random.choices(singlefamily_attached_hp_list, k=math.ceil(TOTAL_BUILDINGS * SINGLEFAMILY_ATTACHED_BUILDINGS["PERCENT_OF_TOTAL"] * SINGLEFAMILY_ATTACHED_BUILDINGS["CHARACTERISTICS"]["HEATPUMP"]))
    singlefamily_attached_nohp_chosen = random.choices(singlefamily_attached_nohp_list, k=math.ceil(TOTAL_BUILDINGS * SINGLEFAMILY_ATTACHED_BUILDINGS["PERCENT_OF_TOTAL"] * (1 - SINGLEFAMILY_ATTACHED_BUILDINGS["CHARACTERISTICS"]["HEATPUMP"])))
    singlefamily_detached_hp_chosen = random.choices(singlefamily_detached_hp_list, k=math.ceil(TOTAL_BUILDINGS * SINGLEFAMILY_DETACHED_BUILDINGS["PERCENT_OF_TOTAL"] * SINGLEFAMILY_DETACHED_BUILDINGS["CHARACTERISTICS"]["HEATPUMP"]))
    singlefamily_detached_nohp_chosen = random.choices(singlefamily_detached_nohp_list, k=math.ceil(TOTAL_BUILDINGS * SINGLEFAMILY_DETACHED_BUILDINGS["PERCENT_OF_TOTAL"] * (1 - SINGLEFAMILY_DETACHED_BUILDINGS["CHARACTERISTICS"]["HEATPUMP"])))

    bldg_list = [multifamily_mid_chosen, multifamily_small_chosen, multifamily_large_chosen, singlefamily_attached_hp_chosen, singlefamily_attached_nohp_chosen, singlefamily_detached_hp_chosen, singlefamily_detached_nohp_chosen]

    bldg_files = []

    for bldg in bldg_list:
        bldg_files.extend(bldg)

    return [bldg_files, [{"multifamily": [{"multifamily_small_chosen": multifamily_small_chosen}, {"multifamily_mid_chosen": multifamily_mid_chosen}, {"multifamily_large_chosen": multifamily_large_chosen}]}, {"singlefamily_attached": [{"singlefamily_attached_hp_chosen": singlefamily_attached_hp_chosen}, {"singlefamily_attached_nohp_chosen": singlefamily_attached_nohp_chosen}]}, {"singlefamily_detached": [{"singlefamily_detached_hp_chosen": singlefamily_detached_hp_chosen}, {"singlefamily_detached_nohp_chosen": singlefamily_detached_nohp_chosen}]}]]

In [161]:
### Commercial

COMMERCIAL_BUILDING_CHARACTERISTICS_FILE = 'DC_upgrade0_agg.xlsx'
COMMERCIAL_BUILDING_CHARACTERISTICS_FILEPATH = DATA_DIR + 'background/' + COMMERCIAL_BUILDING_CHARACTERISTICS_FILE
commercial_building_characteristics = pd.read_excel(COMMERCIAL_BUILDING_CHARACTERISTICS_FILEPATH, sheet_name='building_characteristics')

In [162]:
def construct_neighborhood_commercial():

    warehouse_list = []
    small_office_list = []
    medium_office_list = []
    hospital_list = []
    outpatient_list = []
    primary_school_list = []
    secondary_school_list = []

    small_office_list.extend(commercial_building_characteristics[commercial_building_characteristics['in.comstock_building_type'] == 'SmallOffice']['bldg_id'].tolist())
    #warehouse_list.extend(commercial_building_characteristics[commercial_building_characteristics['in.comstock_building_type'] == 'Warehouse']['bldg_id'].tolist())
    medium_office_list.extend(commercial_building_characteristics[commercial_building_characteristics['in.comstock_building_type'] == 'MediumOffice']['bldg_id'].tolist())
    #hospital_list.extend(commercial_building_characteristics[commercial_building_characteristics['in.comstock_building_type'] == 'Hospital']['bldg_id'].tolist())
    outpatient_list.extend(commercial_building_characteristics[commercial_building_characteristics['in.comstock_building_type'] == 'Outpatient']['bldg_id'].tolist())
    primary_school_list.extend(commercial_building_characteristics[commercial_building_characteristics['in.comstock_building_type'] == 'PrimarySchool']['bldg_id'].tolist())
    #secondary_school_list.extend(commercial_building_characteristics[commercial_building_characteristics['in.comstock_building_type'] == 'SecondarySchool']['bldg_id'].tolist())
   
    total_commercial_building_list = small_office_list + warehouse_list + medium_office_list + hospital_list + outpatient_list + primary_school_list + secondary_school_list
    
    total_commercial_building_chosen = random.choices(total_commercial_building_list, k=2)
   
    # warehouse_chosen = 0
    # small_office_chosen = 0
    # medium_office_chosen = 0
    # hospital_chosen = 0
    # outpatient_chosen = 0
    # primary_school_chosen = 0
    # secondary_school_chosen = 0
    
    small_office_chosen = sum(1 for item in total_commercial_building_chosen if item in small_office_list)
    warehouse_chosen = sum(1 for item in total_commercial_building_chosen if item in warehouse_list)
    medium_office_chosen = sum(1 for item in total_commercial_building_chosen if item in medium_office_list)
    hospital_chosen = sum(1 for item in total_commercial_building_chosen if item in hospital_list)
    outpatient_chosen = sum(1 for item in total_commercial_building_chosen if item in outpatient_list)
    primary_school_chosen = sum(1 for item in total_commercial_building_chosen if item in primary_school_list)
    secondary_school_chosen = sum(1 for item in total_commercial_building_chosen if item in secondary_school_list)

#    commercial_building_list = commercial_building_characteristics['bldg_id'].tolist()
 #   commercial_building_chosen = random.choices(commercial_building_list, k=math.ceil(TOTAL_BUILDINGS * 0.10)) # assuming commercial buildings are 10% of total buildings
    

    return [total_commercial_building_chosen, [{"small_office": small_office_chosen, "warehouse": warehouse_chosen, "medium_office": medium_office_chosen, "hospital": hospital_chosen, "outpatient": outpatient_chosen, "primary_school": primary_school_chosen, "secondary_school": secondary_school_chosen}]]

In [163]:
UPGRADE_NUM = "0"
STATE = "DC"
comstock_bldg_files = construct_neighborhood_commercial()

df_comstock_list = [pd.read_parquet(os.path.join(INPUT_DATA_DIR_COMSTOCK_BUILDINGPROFILES, f"upgrade={UPGRADE_NUM}/state={STATE}/" + str(comstock_filename) + "-0.parquet")) for comstock_filename in comstock_bldg_files[0]]

In [188]:
def resstock_run_summary(resstock_building_characteristics_tot, resstock_bldg_files_tot):
    resstock_building_characteristics_tot_temp = resstock_building_characteristics_tot[resstock_building_characteristics_tot['bldg_id'].isin(resstock_bldg_files_tot[0])]
    resstock_building_characteristics_tot_agg = resstock_building_characteristics_tot_temp.groupby('in.geometry_building_type_height').agg(Count=('bldg_id','count'), Total_ft2=('in.sqft..ft2', 'sum'))
    return resstock_building_characteristics_tot_agg.reset_index().rename(columns={'in.geometry_building_type_height': 'building_type'})

In [189]:
def comstock_run_summary(comstock_building_characteristics_tot, comstock_bldg_files_tot):
    comstock_building_characteristics_tot_temp = comstock_building_characteristics_tot[comstock_building_characteristics_tot['bldg_id'].isin(comstock_bldg_files_tot[0])]
    commstock_building_characteristics_agg = comstock_building_characteristics_tot_temp.groupby('in.comstock_building_type').agg(Count=('bldg_id','count'), Total_ft2=('in.sqft..ft2', 'sum'))
    return commstock_building_characteristics_agg.reset_index().rename(columns={'in.comstock_building_type': 'building_type'})

In [193]:

start_time = time.time()
timestamp_str = str(start_time)

UPGRADE_NUM = 0
STATE = "DC"

random.seed(start_time)

total_merged_buildings_hourly_list = []

if not os.path.exists(f'{OUTPUT_DATA_DIR_SCENARIO_RUNS}{timestamp_str}/'):
    os.makedirs(f'{OUTPUT_DATA_DIR_SCENARIO_RUNS}{timestamp_str}/')
SAMPLE_RUNS = 3
mean = 1
mu = 0.1
ADJUSTMENT_MULTIPLIER = np.random.normal(mean, mu, size=SAMPLE_RUNS)

ADJUSTMENT_MULTIPLIER_ON = True

if not ADJUSTMENT_MULTIPLIER_ON:
    ADJUSTMENT_MULTIPLIER = np.ones(0)

total_merged_buildings_hourly = pd.DataFrame()

building_characteristics_run_summaries = []


for i in range(0,SAMPLE_RUNS):

    resstock_bldg_files = construct_neighborhood_residential()
    comstock_bldg_files = construct_neighborhood_commercial()

    # TODO: Adjust this to save building characteristics along with chosen files

    # chosen_files_df = pd.DataFrame(chosen_files, columns=['filename'])
    # chosen_files_df.to_csv(f'{OUTPUT_DATA_DIR_SCENARIO_RUNS}{timestamp_str}/chosen_files.csv')

    ## TODO Add choices for each run into csv for further analysis.


    # Create a list of DataFrames, one for each file
    df_resstock_list = [pd.read_parquet(os.path.join(INPUT_DATA_DIR_RESSTOCK_BUILDINGPROFILES, f"upgrade={UPGRADE_NUM}/state={STATE}/" + str(resstock_filename) + "-0.parquet")) for resstock_filename in resstock_bldg_files[0]]
    df_comstock_list = [pd.read_parquet(os.path.join(INPUT_DATA_DIR_COMSTOCK_BUILDINGPROFILES, f"upgrade={UPGRADE_NUM}/state={STATE}/" + str(comstock_filename) + "-0.parquet")) for comstock_filename in comstock_bldg_files[0]]

    # Concatenate all DataFrames in the list into a single DataFrame
    resstock_merged_df = pd.concat(df_resstock_list, ignore_index=True)
    comstock_merged_df = pd.concat(df_comstock_list, ignore_index=True)

    print(f"Resstock Run {i+1}: {resstock_bldg_files}")
    resstock_merged_buildings_15m = resstock_merged_df[['timestamp', 'out.electricity.total.energy_consumption..kwh', 'bldg_id', 'in.sqft']]
    resstock_merged_buildings_15m.index = pd.to_datetime(resstock_merged_buildings_15m['timestamp'])

    print(f"Comstock Run {i+1}: {comstock_bldg_files}")
    comstock_merged_buildings_15m = comstock_merged_df[['timestamp', 'out.electricity.total.energy_consumption', 'bldg_id']] #, 'in.sqft..ft2']]
    comstock_merged_buildings_15m.index = pd.to_datetime(comstock_merged_buildings_15m['timestamp'])

    resstock_merged_buildings_hourly = resstock_merged_buildings_15m.resample('h')['out.electricity.total.energy_consumption..kwh'].sum() * ADJUSTMENT_MULTIPLIER[i]
    ax_merged_buildings_hourly = resstock_merged_buildings_hourly.plot(x='timestamp', y='out.electricity.total.energy_consumption..kwh')

    comstock_merged_buildings_hourly = comstock_merged_buildings_15m.resample('h')['out.electricity.total.energy_consumption'].sum() * ADJUSTMENT_MULTIPLIER[i]

    # Resstock hourly
    resstock_merged_buildings_hourly.columns = [f'Resstock Run {i+1}']
    resstock_merged_buildings_hourly.rename({'out.electricity.total.energy_consumption..kwh': f'Resstock_Run {i+1}'}, inplace=True)

    # Comstock hourly
    comstock_merged_buildings_hourly.columns = [f'Comstock Run {i+1}']
    comstock_merged_buildings_hourly.rename({'out.electricity.total.energy_consumption': f'Comstock_Run {i+1}'}, inplace=True)

    #total_merged_buildings_hourly = pd.merge(total_merged_buildings_hourly, merged_buildings_hourly, left_index=True, right_index=True, how='inner') #, suffixes=("", "_DROP"))
    #print(total_merged_buildings_hourly.columns)
    #total_merged_buildings_hourly = total_merged_buildings_hourly.filter(regex='^(?!.*_DROP)')

    resstock_merged_buildings_hourly.to_csv(f'{OUTPUT_DATA_DIR_SCENARIO_RUNS}{timestamp_str}/residential_merged_community_load_profile_{timestamp_str}_run-{i+1}.csv')
    comstock_merged_buildings_hourly.to_csv(f'{OUTPUT_DATA_DIR_SCENARIO_RUNS}{timestamp_str}/comstock_merged_community_load_profile_{timestamp_str}_run-{i+1}.csv')

    #total_merged_buildings_hourly_list.append(resstock_merged_buildings_hourly)

    total_merged_buildings_hourly = pd.concat([total_merged_buildings_hourly, resstock_merged_buildings_hourly, comstock_merged_buildings_hourly], axis=1)

    total_merged_buildings_hourly.groupby(total_merged_buildings_hourly.index).mean().to_csv(f'{OUTPUT_DATA_DIR_SCENARIO_RUNS}{timestamp_str}/total_merged_community_load_profile_{timestamp_str}_run-{i+1}.csv')
    #total_merged_buildings_hourly.to_csv(f'{OUTPUT_DATA_DIR_SCENARIO_RUNS}{timestamp_str}/total_merged_community_load_profile_{timestamp_str}_run-{i+1}.csv')

    #print(resstock_bldg_files[1])

    building_characteristics_run_summaries.append(pd.concat([resstock_run_summary(building_characteristics, resstock_bldg_files), comstock_run_summary(commercial_building_characteristics, comstock_bldg_files)]))
    print(building_characteristics_run_summaries[i])
     #comstock_run_summary(comstock_bldg_files)



#print("total_merged_buildings_hourly_list: ", total_merged_buildings_hourly) #previously total_merged_buildings_hourly_list

#total_merged_buildings_hourly = pd.concat(total_merged_buildings_hourly_list, axis=0)

#print("total_merged_buildings_hourly: ", total_merged_buildings_hourly)

resstock_total_merged_buildings_hourly_columns_runs = ["Resstock Run " + str(i+1) for i in range(SAMPLE_RUNS)]
comstock_total_merged_buildings_hourly_columns_runs = ["Comstock Run " + str(i+1) for i in range(SAMPLE_RUNS)]

total_merged_buildings_hourly_columns_runs = list(itertools.chain.from_iterable(zip(resstock_total_merged_buildings_hourly_columns_runs, comstock_total_merged_buildings_hourly_columns_runs)))

total_merged_buildings_hourly_columns_runs_total = ["Run " + str(i+1) for i in range(SAMPLE_RUNS)]

#print(total_merged_buildings_hourly_columns_runs)

#total_merged_buildings_hourly_columns = ['timestamp'].extend(total_merged_buildings_hourly_columns_runs)
#print(total_merged_buildings_hourly_columns)

total_merged_buildings_hourly.index.name = 'timestamp'
total_merged_buildings_hourly.columns = total_merged_buildings_hourly_columns_runs

total_merged_buildings_hourly.to_csv(f'{OUTPUT_DATA_DIR_SCENARIO_RUNS}{timestamp_str}/merged_community_load_profile_total_compiled-runs.csv')

ax_merged_buildings_hourly.get_figure().savefig(f'{OUTPUT_DATA_DIR_SCENARIO_RUNS}{timestamp_str}/merged_community_load_profile_total.png')
plt.close(ax_merged_buildings_hourly.get_figure())

#total_merged_buildings_hourly = pd.concat(total_merged_buildings_hourly, axis=0) # previously total_merged_buildings_hourly_list
#total_merged_buildings_hourly.to_csv(f'{OUTPUT_DATA_DIR_SCENARIO_RUNS}{timestamp_str}/merged_community_load_profile_{timestamp_str}_total.csv')
#print(f"Total Merged Buildings Hourly: ", total_merged_buildings_hourly)
#total_merged_buildings_hourly_average = total_merged_buildings_hourly.groupby(total_merged_buildings_hourly.index).mean()
total_merged_buildings_hourly_average = total_merged_buildings_hourly.mean()
total_merged_buildings_hourly_average["Resstock Average"] = total_merged_buildings_hourly.filter(like="Resstock Run ").mean(axis=0)
total_merged_buildings_hourly_average["Comstock Average"] = total_merged_buildings_hourly.filter(like="Comstock Run ").mean(axis=0)
# print(f"Total Merged Buildings Hourly Average: ", total_merged_buildings_hourly_average)

total_merged_buildings_hourly_average.to_csv(f'{OUTPUT_DATA_DIR_SCENARIO_RUNS}{timestamp_str}/merged_community_load_profile_{timestamp_str}_hourly_average.csv')

# ax_total_merged_buildings_hourly_average = total_merged_buildings_hourly_average.plot(x='timestamp', y='out.electricity.total.energy_consumption..kwh', figsize=(15,5))
# ax_total_merged_buildings_hourly_average.get_figure().savefig(f'{OUTPUT_DATA_DIR_SCENARIO_RUNS}{timestamp_str}/merged_community_load_profile_{timestamp_str}_average.png')
# ax_total_merged_buildings_hourly_average.set_title('Average Community Load Profile over ' + str(SAMPLE_RUNS) + ' runs')

end_time = time.time() 

time_diff = end_time - start_time
print(f"Time taken for processing {SAMPLE_RUNS} runs: {time_diff/60.0} minutes")



Resstock Run 1: [[189288, 84246, 446869, 163536, 404020, 290616, 205901, 151415, 478454, 478454, 193877, 428288, 508599, 460255, 500637, 515853, 285780, 375766, 389636, 41019, 207066, 488380, 246230, 470742, 470742, 424865, 473864, 485891, 362036, 434049, 341113, 15458, 389636, 533139, 77783, 142631, 318261, 321251, 174255, 538578, 50348, 11250, 50348, 383014, 115474, 320552, 86497, 476266, 508809, 428770, 329981, 54339, 278753, 265230, 113380, 103662, 230055, 459428, 390947, 91623, 243742, 17921, 354298, 544448, 122912, 461232, 138905, 113380, 302335, 11140, 256013, 499953, 126506, 499953, 256013, 428770, 132586, 134730, 143013, 52913, 146481, 491258, 345773, 381838, 147618, 381838, 228570, 502683, 340759, 381838, 435918, 176025, 227077, 147618, 227077, 323093, 402939, 91915, 413833, 483401], [{'multifamily': [{'multifamily_small_chosen': [193877, 428288, 508599, 460255, 500637, 515853, 285780, 375766, 389636, 41019, 207066, 488380, 246230, 470742, 470742, 424865, 473864, 485891, 3620

/Users/egeriicw/Personal-Projects/Microgrid Modeling/Community Load Profiles/.venv/lib/python3.14/site-packages/pandas/core/indexes/base.py:681: FutureWarning: Dtype inference on a pandas object (Series, Index, ExtensionArray) is deprecated. The Index constructor will keep the original dtype in the future. Call `infer_objects` on the result to get the old behavior.
  result = cls(*args, **kwargs)


                             building_type  Count  Total_ft2
0            Multi-Family with 2 - 4 Units      6       7117
1  Multi-Family with 5+ Units, 1-3 Stories     23      17955
2  Multi-Family with 5+ Units, 4-7 Stories     10       9498
3                   Single-Family Attached     33      44503
4                   Single-Family Detached     16      23878
0                               Outpatient      1       2000
1                            PrimarySchool      1      21000
Resstock Run 2: [[518393, 127521, 156961, 403995, 506670, 29827, 365413, 488816, 454643, 69514, 465487, 8092, 270384, 516925, 468866, 310785, 315971, 66513, 393025, 452686, 424865, 404962, 5985, 340243, 353925, 333548, 533139, 119716, 489178, 50348, 55684, 63061, 428288, 142631, 80197, 461646, 461646, 85483, 498453, 54092, 450214, 67922, 312386, 395734, 378668, 91623, 170206, 520990, 523525, 115474, 499953, 378656, 91623, 133586, 138905, 407562, 212289, 468700, 19496, 50488, 326693, 313152, 11140, 392834, 1

/Users/egeriicw/Personal-Projects/Microgrid Modeling/Community Load Profiles/.venv/lib/python3.14/site-packages/pandas/core/indexes/base.py:681: FutureWarning: Dtype inference on a pandas object (Series, Index, ExtensionArray) is deprecated. The Index constructor will keep the original dtype in the future. Call `infer_objects` on the result to get the old behavior.
  result = cls(*args, **kwargs)


                             building_type  Count  Total_ft2
0            Multi-Family with 2 - 4 Units      6       4929
1  Multi-Family with 5+ Units, 1-3 Stories     24      19213
2  Multi-Family with 5+ Units, 4-7 Stories     12      10691
3                   Single-Family Attached     37      50137
4                   Single-Family Detached     16      24348
0                               Outpatient      1       2000
1                            PrimarySchool      1      35000
Resstock Run 3: [[30005, 523635, 522914, 478454, 346974, 174382, 192335, 448533, 478454, 540108, 358471, 220514, 385251, 89502, 275762, 248235, 77991, 248235, 499417, 548589, 289584, 473864, 515853, 327785, 323110, 424865, 460255, 546388, 498453, 13316, 343760, 387362, 89154, 260281, 98701, 443445, 137628, 469987, 127345, 247471, 463812, 490543, 207205, 490543, 170206, 354298, 316879, 228245, 543249, 347542, 50488, 127903, 108248, 95403, 107895, 119281, 219148, 95403, 32003, 358000, 463159, 253592, 307761, 

/Users/egeriicw/Personal-Projects/Microgrid Modeling/Community Load Profiles/.venv/lib/python3.14/site-packages/pandas/core/indexes/base.py:681: FutureWarning: Dtype inference on a pandas object (Series, Index, ExtensionArray) is deprecated. The Index constructor will keep the original dtype in the future. Call `infer_objects` on the result to get the old behavior.
  result = cls(*args, **kwargs)


                             building_type  Count  Total_ft2
0            Multi-Family with 2 - 4 Units     10      11212
1  Multi-Family with 5+ Units, 1-3 Stories     21      21509
2  Multi-Family with 5+ Units, 4-7 Stories      9       7046
3                   Single-Family Attached     33      40586
4                   Single-Family Detached     18      23289
0                               Outpatient      1      21000
1                              SmallOffice      1      21000
Time taken for processing 3 runs: 0.3470636487007141 minutes


In [ ]:
## Run aggregations

# total_merged_buildings_daily_total = total_merged_buildings_hourly_average.resample('D').sum()
# total_merged_buildings_daily_total.to_csv(f'{OUTPUT_DATA_DIR_SCENARIO_RUNS}{timestamp_str}/merged_community_load_profile_{timestamp_str}_total_daily.csv')
# total_merged_buildings_monthly_total = total_merged_buildings_hourly_average.resample('ME').sum()
# total_merged_buildings_monthly_total.to_csv(f'{OUTPUT_DATA_DIR_SCENARIO_RUNS}{timestamp_str}/merged_community_load_profile_{timestamp_str}_total_monthly.csv')

In [ ]:
# ## Create Summary File

# workbook = xw.Workbook(f'{OUTPUT_DATA_DIR_SCENARIO_RUNS}{timestamp_str}/output.xlsx')
# worksheet = workbook.add_worksheet('Overview')

# row = 0
# col = 0

# worksheet.write(row, col, 'TimeStart (timestamp)')
# worksheet.write(row+1, col, str(start_time))
# worksheet.write(row, col+1, 'TimeEnd (timestamp)')
# worksheet.write(row+1, col+1, str(end_time))
# worksheet.write(row, col+2, 'TimeDiff (minutes)')
# worksheet.write(row+1, col+2, str(time_diff/60.0))

# workbook.close()

# # # Open the existing file in append mode
# # with pd.ExcelWriter(f'{DIR}{timestamp_str}/output.xlsx', mode='a', engine='openpyxl', if_sheet_exists='new') as writer: